In [259]:
#import thinkplot
#import thinkstats2
import pandas as pd
import numpy as np
import math
import scipy.stats as ss
import thinkplot
import thinkstats2
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (15,5)

# Assignment 3 - Basic Predictions and Regression

## Questions

### Part 1 - Election Prediction

Suppose you are looking at an election in a fictional province. There are 7 total elctoral districts, and the winner in each district is determined by a first-past-the-post system (what we have in Canada - the most votes wins, regardless of share). There are two parties - the Purples and the Yellows. Whoever controls the most seats will be the ruling party - so in our 2 party scenario, the party who wins 4 or more of the districts will govern. There is an election every year, they love voting. 

Recent polling indicating the expected vote share in each district is shown in the "dist_polls" table below. These values are a composite of several polls that the experts have combined and weighted. The "Purple" values show the expected vote share of the Purple party, along with the variance of that expectation and the number of polls that were combined to get that result.  

As well, research has shown that the vote distribution is impacted by voter turnout. In general, the more people vote, the more the vote split shifts towards the Yellow party. We have data on past elections and the results, we expect that the turnout will be in line with the past elections - or more specifically, we have no reason to expect it to differ. This impact is measured in the table in code below - that table shows the voter turnout, in a percentage, as well as the change in the Yellow party's vote share (also in percentage) as compared to the polling averages. For example, if one row showed "52" and ".8", that would mean that voter turnout was 52%, and the Yellow party got .8% higher of a vote share than the polling showed. 

<b>What is the probability that the Purple Party controls the government after the election?</b>

<b>Note:</b> the errors and confidence intervals are not totally trivial. As part of the written answer, offer an evaluation of your confidence in the prediction, and why you think that. This is not a question with one specific error, your estimation will have some expected errors, somewhere. You may not have the tools to calculate it all the way through, that's fine. 

### Question 1 - Your Answer in English

Please fill in (and extend if required) the list here to explain what you did. There are multiple reasonable things you could do to approach this, so please note what you did here. For most people I assume this will be about 3-5 statements - you don't need to explain the internals of things we covered (e.g. if there's a hypothesis test, you don't need to explain how that works), just how you structured your approach to the problem. 

<ul> 
<li> use Linear regression to predict on average of Yellow party if they had a 60% voter turnout.
<li> calculate the average of the Purple party for 7 districts, and use (1 - average) to add a column 'district of yellow'.
<li> use the yellow party's average minus the yellow party prediction of 60% turnout.
<li> define simulation and 1000 trials to predict the Purple party's vote.
<li> the probability of the Purple party wins would be 1/7 district.
<li> <b> What do you think about the error/accuracy:</b>
<li> Errors are similarly low, between 1.5% to 1.6%.
</ul>

##### Setup Poll Data

The dataframe "dist_polls" contains all of the polls for each seat. Each value is expressed as expected vote share (as a ratio) for the <b>Purple</b> party. The Yellow party can be safely assumed to get the rest of the votes. 

In [260]:
# Please don't edit this part. 
# Setup polling data. 
districts = [1,2,3,4,5,6,7]
dist_polls = pd.DataFrame(districts, columns={"district"})

dist_polls["Poll_1"] = [.55, .49, .51, .6, .41, .46, .54]
dist_polls["Poll_2"] = [.53, .51, .51, .62, .44, .48, .53]
dist_polls["Poll_3"] = [.51, .49, .53, .61, .42, .46, .52]
dist_polls["Poll_4"] = [.47, .48, .51, .54, .45, .45, .51]
dist_polls["Poll_5"] = [.61, .52, .49, .73, .44, .51, .53]
dist_polls["Poll_6"] = [.54, .45, .51, .61, .47, .52, .52]
dist_polls["Poll_7"] = [.55, .47, .5, .56, .47, .46, .56]
dist_polls["Poll_8"] = [.53, .49, .51, .55, .43, .49, .55]
dist_polls["Poll_9"] = [.57, .39, .52, .57, .53, .43, .53]


dist_polls.head(7)

,district,Poll_1,Poll_2,Poll_3,Poll_4,Poll_5,Poll_6,Poll_7,Poll_8,Poll_9
0,1,0.55,0.53,0.51,0.47,0.61,0.54,0.55,0.53,0.57
1,2,0.49,0.51,0.49,0.48,0.52,0.45,0.47,0.49,0.39
2,3,0.51,0.51,0.53,0.51,0.49,0.51,0.50,0.51,0.52
3,4,0.60,0.62,0.61,0.54,0.73,0.61,0.56,0.55,0.57
4,5,0.41,0.44,0.42,0.45,0.44,0.47,0.47,0.43,0.53
5,6,0.46,0.48,0.46,0.45,0.51,0.52,0.46,0.49,0.43
6,7,0.54,0.53,0.52,0.51,0.53,0.52,0.56,0.55,0.53


##### Setup Turnout Data

The dataframe "past_vte_table" shows the voter turnout, along with the impact on the votes counted for the <b>Yellow party</b>, all expressed as percentages. For example, if in one row the turnout is .45 and the Yellow_improvement is -.04, that means that 45% of the populace turned out to vote, and the Yellow party got 4% fewer votes than polling indicated. 

In [261]:
# Please don't edit this part. 
# Setup vote data. 
voter_turnout_history = [.53, .51, .48, .55, .54, .59, .49, .57, .56]
past_vote_table = pd.DataFrame(voter_turnout_history, columns={"voter_turn_percentage"})
past_vote_table["Yellow_improvement"] = [.012, .023, -.017, .031, .030, -.004, -.03, .042, .029]
past_vote_table["year"] = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
past_vote_table.head()

,voter_turn_percentage,Yellow_improvement,year
0,0.53,0.012,2013
1,0.51,0.023,2014
2,0.48,-0.017,2015
3,0.55,0.031,2016
4,0.54,0.030,2017


## Start Your Work

### Part 1 - Election

In [262]:
y = np.array(past_vote_table['Yellow_improvement']).reshape(-1,1)
x = np.array(past_vote_table['voter_turn_percentage']).reshape(-1,1)

x.shape, y.shape

((9, 1), (9, 1))

In [263]:
vModel = LinearRegression().fit(x,y)
y_predict = vModel.predict(np.array(0.6).reshape(-1, 1))
y_predict

array([[0.03775359]])

In [264]:
rsqr = vModel.score(x,y)
print('Coefficient of determination:', rsqr)
print('Intercept:', vModel.intercept_[0])
print('Slope:', vModel.coef_[0][0])

Coefficient of determination: 0.33225372047195967
Intercept: -0.1937453798767969
Slope: 0.3858316221765917


In [265]:
dist_polls['dis_mean'] = dist_polls.iloc[ : , 1 : 9 ].mean(axis = 1)
dist_polls.drop(columns = ['district'], axis = 1, inplace = True)
dist_polls.T

,0,1,2,3,4,5,6
Poll_1,0.55000,0.4900,0.51000,0.6000,0.41000,0.46000,0.5400
Poll_2,0.53000,0.5100,0.51000,0.6200,0.44000,0.48000,0.5300
Poll_3,0.51000,0.4900,0.53000,0.6100,0.42000,0.46000,0.5200
Poll_4,0.47000,0.4800,0.51000,0.5400,0.45000,0.45000,0.5100
Poll_5,0.61000,0.5200,0.49000,0.7300,0.44000,0.51000,0.5300
Poll_6,0.54000,0.4500,0.51000,0.6100,0.47000,0.52000,0.5200
Poll_7,0.55000,0.4700,0.50000,0.5600,0.47000,0.46000,0.5600
Poll_8,0.53000,0.4900,0.51000,0.5500,0.43000,0.49000,0.5500
Poll_9,0.57000,0.3900,0.52000,0.5700,0.53000,0.43000,0.5300
dis_mean,0.53625,0.4875,0.50875,0.6025,0.44125,0.47875,0.5325


In [266]:
dist_polls.head(7)

,Poll_1,Poll_2,Poll_3,Poll_4,Poll_5,Poll_6,Poll_7,Poll_8,Poll_9,dis_mean
0,0.55,0.53,0.51,0.47,0.61,0.54,0.55,0.53,0.57,0.53625
1,0.49,0.51,0.49,0.48,0.52,0.45,0.47,0.49,0.39,0.48750
2,0.51,0.51,0.53,0.51,0.49,0.51,0.50,0.51,0.52,0.50875
3,0.60,0.62,0.61,0.54,0.73,0.61,0.56,0.55,0.57,0.60250
4,0.41,0.44,0.42,0.45,0.44,0.47,0.47,0.43,0.53,0.44125
5,0.46,0.48,0.46,0.45,0.51,0.52,0.46,0.49,0.43,0.47875
6,0.54,0.53,0.52,0.51,0.53,0.52,0.56,0.55,0.53,0.53250


In [267]:
dist_polls['dis_yellow'] = 1 - (dist_polls['dis_mean'])
dist_polls.head(7)

,Poll_1,Poll_2,Poll_3,Poll_4,Poll_5,Poll_6,Poll_7,Poll_8,Poll_9,dis_mean,dis_yellow
0,0.55,0.53,0.51,0.47,0.61,0.54,0.55,0.53,0.57,0.53625,0.46375
1,0.49,0.51,0.49,0.48,0.52,0.45,0.47,0.49,0.39,0.48750,0.51250
2,0.51,0.51,0.53,0.51,0.49,0.51,0.50,0.51,0.52,0.50875,0.49125
3,0.60,0.62,0.61,0.54,0.73,0.61,0.56,0.55,0.57,0.60250,0.39750
4,0.41,0.44,0.42,0.45,0.44,0.47,0.47,0.43,0.53,0.44125,0.55875
5,0.46,0.48,0.46,0.45,0.51,0.52,0.46,0.49,0.43,0.47875,0.52125
6,0.54,0.53,0.52,0.51,0.53,0.52,0.56,0.55,0.53,0.53250,0.46750


In [268]:
y_predict = 0.03775359

d1 = 0.46375 - 0.03775359
d2 = 0.51250 - 0.03775359
d3 = 0.49125 - 0.03775359
d4 = 0.39750 - 0.03775359
d5 = 0.55875 - 0.03775359
d6 = 0.52125 - 0.03775359
d7 = 0.46750 - 0.03775359

In [269]:
dlist = []
dlist.append(d1)
dlist.append(d2)
dlist.append(d3)
dlist.append(d4)
dlist.append(d5)
dlist.append(d6)
dlist.append(d7)

In [270]:
def oneVote(probCalc):
    vote = np.random.binomial(n=1, p=probCalc)
    return vote

def getSample(voteProb, n=1000):
    vote_list = []
    for i in range(n):
        vote_list.append(oneVote(voteProb))
    return vote_list

#Make a CDF of the means of the analytical dist's
def getSamples(voteProb, n=1000, samples=100, ciLow=2.5, ciHi=97.5):
    meanList = []
    for i in range(samples):
        meanList.append(np.mean(getSample(voteProb, n)))
    muList = [voteProb] * samples
    cdf = thinkstats2.Cdf(meanList) 
    ci = cdf.Percentile(ciLow), cdf.Percentile(ciHi)
    stderr = mean_squared_error(meanList, muList, squared=False)
    return meanList, stderr, cdf, ci

In [271]:
def pred(d) :
    trial = 1000
    means, err, cdfFin, ciFin = getSamples(d, n=1000, samples=trial, ciLow=5, ciHi=95)

    pWins = 0 
    for i in range(len(dlist)):
        if dlist[i] > .5000:
            pWins = pWins + 1
    print(pWins/trial)
    print('probability of Purple party %.1f%% ' % ((1-cdfFin.Prob(.50))*100))
    print('they will get between %.1f%% and %.1f%% percent of the vote' % (ciFin[0]*100, ciFin[1]*100))
    print('Error: ', err)

In [282]:
pre1 = pred(d1)
pre2 = pred(d2)
pre3 = pred(d3)
pre4 = pred(d4)
pre5 = pred(d5)
pre6 = pred(d6)
pre7 = pred(d7)

0.001
probability of Purple party 0.0% 
they will get between 40.0% and 45.2% percent of the vote
Error:  0.015526448455719039
0.001
probability of Purple party 7.3% 
they will get between 44.7% and 50.3% percent of the vote
Error:  0.01641130380829323
0.001
probability of Purple party 0.3% 
they will get between 42.7% and 48.1% percent of the vote
Error:  0.016220650712844415
0.001
probability of Purple party 0.0% 
they will get between 33.5% and 38.4% percent of the vote
Error:  0.015105843535800972
0.001
probability of Purple party 92.2% 
they will get between 49.6% and 54.7% percent of the vote
Error:  0.015128623265456128
0.001
probability of Purple party 15.4% 
they will get between 45.7% and 51.1% percent of the vote
Error:  0.016150466808365016
0.001
probability of Purple party 0.0% 
they will get between 40.3% and 45.6% percent of the vote
Error:  0.01633460733437141


### Part 2 - Regression

<b>Use the data provided to try to predict the wage. </b>

The data is from FIFA rankings for players. You don't need to know anything about soccer or video games for this, so if these values are meaningless to you, just treat them as numbers and you'll be fine. All of the features are rankings are evaluations of how good different soccar players are at different skills.

#### Answer in English

Please fill in (and extend if required) the list here to explain what you did. There are multiple reasonable things you could do to approach this, so please note what you did here. For most people I assume this will be about 3-5 statements - you don't need to explain the internals of things we covered (e.g. if there's a hypothesis test, you don't need to explain how that works), just how you structured your approach to the problem. 

<ul>
<li> looking for errors, nulls and outliers(values too large or too small).
<li> make a new dataframe into an array and Split data.
<li> run LinearRegression and predict RMSE = 10671, R2 = 0.354.
<li> add Y value, predictions and residual into a new dataframe.
</ul>

In [273]:
df = pd.read_csv("players_20_2.csv")
df.head()

,wage_eur,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,...,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle
0,565000,88,95,70,92,88,97,93,94,92,...,94,48,40,94,94,75,96,33,37,26
1,405000,84,94,89,83,87,89,81,76,77,...,93,63,29,95,82,85,95,28,32,24
2,290000,87,87,62,87,87,96,88,87,81,...,84,51,36,87,90,90,94,27,26,29
3,125000,13,11,15,43,13,12,13,14,40,...,12,34,19,11,65,11,68,27,12,18
4,470000,81,84,61,89,83,95,83,79,83,...,80,54,41,87,89,88,91,34,27,22


In [274]:
df.describe()

,wage_eur,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,...,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle
count,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,...,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000,18278.000000
mean,9456.942773,49.718405,45.590218,52.221468,58.748003,42.809388,55.596509,47.325692,42.708885,52.768848,...,46.812945,55.742149,46.380239,50.072163,53.609749,48.383357,58.528778,46.848889,47.640333,45.606631
std,21351.714095,18.325403,19.594609,17.428429,14.679653,17.701815,18.929118,18.422991,17.433612,15.246454,...,19.322343,17.318157,20.775812,19.594022,13.955626,15.708099,11.880840,20.091287,21.585641,21.217734
min,0.000000,5.000000,2.000000,5.000000,7.000000,3.000000,4.000000,6.000000,4.000000,8.000000,...,4.000000,9.000000,3.000000,2.000000,9.000000,7.000000,12.000000,1.000000,5.000000,3.000000
25%,1000.000000,38.000000,30.000000,44.000000,54.000000,30.000000,50.000000,34.000000,31.000000,43.000000,...,32.000000,44.000000,25.000000,39.000000,44.000000,39.000000,51.000000,29.000000,27.000000,24.000000
50%,3000.000000,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,49.000000,41.000000,56.000000,...,51.000000,58.000000,52.000000,55.000000,55.000000,49.000000,60.000000,52.000000,55.000000,52.000000
75%,8000.000000,64.000000,62.000000,64.000000,68.000000,56.000000,68.000000,62.000000,56.000000,64.000000,...,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.000000
max,565000.000000,93.000000,95.000000,93.000000,92.000000,90.000000,97.000000,94.000000,94.000000,92.000000,...,94.000000,95.000000,92.000000,95.000000,94.000000,92.000000,96.000000,94.000000,92.000000,90.000000


In [275]:
df.isnull().sum().sort_values()

wage_eur                      0
defending_marking             0
mentality_composure           0
mentality_penalties           0
mentality_vision              0
mentality_positioning         0
mentality_interceptions       0
mentality_aggression          0
power_long_shots              0
power_strength                0
power_stamina                 0
power_jumping                 0
power_shot_power              0
movement_balance              0
movement_reactions            0
movement_agility              0
movement_sprint_speed         0
movement_acceleration         0
skill_ball_control            0
skill_long_passing            0
skill_fk_accuracy             0
skill_curve                   0
skill_dribbling               0
attacking_volleys             0
attacking_short_passing       0
attacking_heading_accuracy    0
attacking_finishing           0
attacking_crossing            0
defending_standing_tackle     0
defending_sliding_tackle      0
dtype: int64

In [276]:
df3=df[df["wage_eur"] > 5000]
df3=df[df["wage_eur"] < 100000]
df3.describe()

,wage_eur,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,...,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle
count,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,...,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.000000,18105.00000
mean,7922.618061,49.540458,45.409003,52.087876,58.559072,42.608727,55.414803,47.122066,42.532781,52.578017,...,46.618172,55.612980,46.235239,49.881359,53.407291,48.235294,58.309749,46.714223,47.517592,45.49931
std,12748.065686,18.226236,19.486583,17.357776,14.579053,17.545066,18.840642,18.288698,17.306730,15.144739,...,19.211724,17.274977,20.697420,19.478694,13.814588,15.616693,11.706434,20.024757,21.515977,21.14326
min,0.000000,5.000000,2.000000,5.000000,7.000000,3.000000,4.000000,6.000000,4.000000,8.000000,...,4.000000,9.000000,3.000000,2.000000,9.000000,7.000000,12.000000,1.000000,5.000000,3.00000
25%,1000.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,34.000000,30.000000,43.000000,...,32.000000,44.000000,25.000000,39.000000,44.000000,39.000000,51.000000,29.000000,27.000000,24.00000
50%,3000.000000,54.000000,49.000000,55.000000,62.000000,43.000000,61.000000,49.000000,41.000000,56.000000,...,51.000000,58.000000,52.000000,55.000000,55.000000,49.000000,59.000000,52.000000,55.000000,52.00000
75%,8000.000000,63.000000,62.000000,64.000000,68.000000,56.000000,68.000000,62.000000,56.000000,64.000000,...,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.00000
max,99000.000000,91.000000,89.000000,93.000000,91.000000,88.000000,92.000000,94.000000,92.000000,89.000000,...,90.000000,95.000000,87.000000,91.000000,89.000000,91.000000,92.000000,92.000000,87.000000,90.00000


In [277]:
# Setup
# #Make that df into an array. 
y = np.array(df3["wage_eur"]).reshape(-1,1)
x = np.array(df3.drop(columns={"wage_eur"}))
print("X shape", x.shape)
print("X shape", y.shape)

X shape (18105, 29)
X shape (18105, 1)


In [278]:
#Split data
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=.3)

In [279]:
#Generate model 
model = LinearRegression().fit(xTrain,yTrain)

In [280]:
tmp = model.predict(xTest)
rmse = mean_squared_error(tmp, yTest, squared=False)
rsqr = model.score(xTest, yTest)

print("RMSE:", rmse)
print('R-squared:', rsqr)

RMSE: 10067.828096684305
R-squared: 0.3536301243101343


In [281]:
#Get Residuals and picture them in a DF for easy reading. 
tmp1 = pd.DataFrame(yTest, columns={"Yvalue"})
tmp2 = pd.DataFrame(tmp, columns={"Predictions"})
tmp3 = pd.DataFrame((yTest-tmp), columns={"Residual"})
resFrame = pd.concat([tmp1,tmp2,tmp3], axis=1)
resFrame.head()

,Yvalue,Predictions,Residual
0,10000,17806.318791,-7806.318791
1,8000,7048.404173,951.595827
2,3000,4212.733021,-1212.733021
3,2000,9239.434073,-7239.434073
4,5000,7698.277125,-2698.277125
